In [1]:
import src.models_and_optimizers as model_utils
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
import os
import torch
from tqdm import tqdm
from torch.cuda.amp import autocast
from transformers import EsmTokenizer, EsmModel
import esm as esmlib

/home/fosterb/anaconda3/envs/rla_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-18 19:01:54.172266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 19:01:54.182813: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 19:01:54.186020: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 19:01:54.194905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFl

In [2]:
## GENERAL SETUP (CHANGE PATHS AS NEEDED)
ROOT = "model_weights"
model_dir = "version_0/" 
dev = 'cuda:0'
CLIP_MODE = False
root_path = os.path.join(ROOT, model_dir)
path = os.path.join(root_path, "checkpoints/checkpoint_best.pt")
data_root = "/home/fosterb/rla_test/wds" #
args_path = os.path.join(ROOT, model_dir, [u for u in os.listdir(os.path.join(ROOT, model_dir)) if u.endswith('.pt')][0])

backwards_compat = {
    'masked_rate': -1,
    'masked_mode': 'MASK',
    'lm_only_text': 1,
    'lm_weight': 1,
    'resid_weight': 1,
    'language_head': False,
    'language_head_type': 'MLP',
    'zip_enabled': False,
    'num_mutations': False,
}
hparams = torch.load(args_path)
args_dict = hparams['args']
args_dict['data_root'] = data_root
args_dict['train_wds_path'] = 'wds_'
args_dict['val_wds_path'] = 'wds_'
args_dict['batch_size'] = 1
args_dict['num_workers'] = 1
args_dict['blacklist_file'] = ''
for k in backwards_compat.keys():
    if k not in args_dict:
        args_dict[k] = backwards_compat[k]
args = SimpleNamespace(**args_dict)

print(vars(args))

coordinator_params = data_utils.get_coordinator_params(args.coordinator_hparams)
coordinator_params['num_positional_embeddings'] = args.gnn_num_pos_embs
coordinator_params['zero_out_pos_embs']= args.gnn_zero_out_pos_embs
coordinator_params['clip_mode'] = True

{'num_workers': 1, 'batch_size': 1, 'exp_name': '11_13_new_blacklist', 'epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'momentum': 0.9, 'lr_peak_epoch': 2, 'label_smoothing': 0.0, 'disable_logging': 0, 'data_root': '/home/fosterb/rla_test/wds', 'eval_epochs': 2, 'out_dir': 'runs/', 'only_non_bn_weight_decay': False, 'opt': 'ADAM', 'lr_scheduler': 'cosine', 'mixed_precision': 1, 'max_seq_len': 1024, 'self_supervised': 1, 'burn_in': 0, 'max_coord_len': 2000, 'freeze_llm': 0, 'freeze_text_proj': 0, 'finetune_from': '', 'num_mutations': -1, 'mutation_fwd_method': 'all', 'masked_rate': -1.0, 'masked_mode': 'MASK', 'lm_only_text': 1, 'lm_weight': 1.0, 'resid_weight': 1.0, 'zip_enabled': 0, 'zip_train_format_string': '', 'cath_info_dict': '', 'arch': 'facebook/esm2_t30_150M_UR50D', 'coordinator_hparams': 'terminator_configs/coordinator_broken_merge.json', 'gnn_checkpoint': '', 'gnn_num_pos_embs': 16, 'gnn_zero_out_pos_embs': False, 'language_head': 0, 'language_head_type': 'MLP', 'train_wds

/tmp/ipykernel_255123/613479242.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hparams = torch.load(args_path)


In [3]:
## LOAD MODEL (NO CHANGES NEEDED)
args_dict['arch']= 'esm_model_150'
trained_model = model_utils.load_model(path, args_dict['arch'], dev)
tokenizer = EsmTokenizer.from_pretrained(args_dict['arch'])
esm_arch = 'esm_model_150'
esm_model = EsmModel.from_pretrained(args_dict['arch']) 
esm_model = esm_model.to(dev)
esm_model.eval()
esm, alphabet = esmlib.pretrained.esm2_t30_150M_UR50D()
# esm, alphabet = esmlib.pretrained.esm1v_t33_650M_UR90S_1()
esm = esm.eval().cuda()

/home/fosterb/rla_test/src/models_and_optimizers.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path, map_location=torch.device(device))


loading state dict from model_weights/version_0/checkpoints/checkpoint_best.pt


Some weights of EsmModel were not initialized from the model checkpoint at esm_model_150 and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


building model based on path
ZERO OUT POS EMB False
GNN Potts Model Encoder hidden dimensionality is 128
freeze_llm False
use text proj:  True
args: 
320 True


/home/fosterb/anaconda3/envs/rla_new/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of EsmModel were not initialized from the model checkpoint at esm_model_150 and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
## LOAD UTIL FUNCTIONS (NO CHANGES NEEDED)
%run 2023_12_11_rla_utils.ipynb

/home/fosterb/anaconda3/envs/rla_new/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/tmp/ipykernel_255123/2500981057.py:21: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [5]:
trained_model = trained_model.eval()

In [11]:
## DEFINE RLA SETTING CALCULATIONS (SOME CHANGES POSSIBLE)
seq_mask=None # 'peptide' to mask peptide sequence, 'protein' to mask protein sequence
struct_mask=None # 'peptide' to mask peptide structure, 'protein' to mask protein structure
top_k = 30 # Num neighbors, probably want to keep at 30 but can experiment
focus = True # RLA calculation setting that limits RLA score to interface, almost certainly want to keep True
remove_far = True # Removes residues too far from the interface, likely want to keep True
threshold = 1 # Threshold for remove_far calculation, likely want to keep at 1
weight_dists = False # Weights RLA score per residue by distance from interface, likely want to keep False
pep_weight = 1 # Weight of peptide residues relative to protein residues, likely want to keep at 1


In [12]:
## DATASETS TO EVALUATE (CHANGE AS NEEDED)
design_sets = ["4cay", "4cay_new", "4cay_supercloud"]

In [13]:
args.batch_size=1

In [14]:
## PERFORMS RLA CALCULATION (NO CHANGES NEEDED)
if CLIP_MODE:
    feature_getter = get_text_and_image_features_clip
else:
    feature_getter = get_text_and_image_features
    
torch.multiprocessing.set_sharing_strategy('file_system')
nclash_dict, Fnat_dict, Fnonnat_dict, LRMS_dict, iRMSDbb_dict, irmsdsc_dict, distance_dict, theta_dict, class_dict = {}, {}, {}, {}, {}, {}, {}, {}, {}
dicts = [nclash_dict, Fnat_dict, Fnonnat_dict, LRMS_dict, iRMSDbb_dict, irmsdsc_dict, distance_dict, theta_dict, class_dict]
nclash_data, Fnat_data, Fnonnat_data, LRMS_data, iRMSDbb_data, irmsdsc_data, distance_data, theta_data, class_data = {}, {}, {}, {}, {}, {}, {}, {}, {}
data_dicts = [nclash_data, Fnat_data, Fnonnat_data, LRMS_data, iRMSDbb_data, irmsdsc_data, distance_data, theta_data, class_data]
args.batch_size = 1
args.zip_enabled = False
args.num_mutations = 0
args.distributed = 0
plot_scores = []
plot_weights = []
plot_pep_mask = []
plot_indices = []
plot_X = []
plot_seq = []
paired_res_scores = {}
scores_stats = {'models': [], 'seqs': [], 'rla_scores': []}
# result_types = ['nclash', 'fnat', 'fnonnat', 'lrmsd', 'irmsdbb', 'irmsdsc', 'distance', 'theta', 'classification']
for design_set in design_sets:
    print(f'running on {design_set}.')
    args.train_wds_path = f"{design_set}.wds"
    args.val_wds_path = f"{design_set}.wds"
    train_loader, val_loader, train_len, val_len = get_wds_loaders(args, coordinator_params, gpu=None, shuffle_train=False, val_only=True, return_count=False)
    lens = {}
    for i, b in tqdm(enumerate(val_loader), total=val_len):
        lens[b[0]['pdb_id'][0]] = len(b[0]['string_sequence'][0])
    MAX_LEN = max(lens.values())
    
    for i, batch in enumerate(tqdm(val_loader, total=val_len)):
        model = batch[0]['pdb_id'][0]
        pep_seq = batch[0]['string_sequence'][0][:batch[1]['seq_lens'][0][0]]
        chain_lens = torch.zeros(batch[1]['coords'][0].shape[1]).to(device = batch[1]['coords'][0].device)
        chain_lens[batch[1]['seq_lens'][0][0]:] = 1
        chain_lens_mask = torch.ones(batch[1]['coords'][0].shape[1]).unsqueeze(0).to(dtype=torch.bool, device = batch[1]['coords'][0].device)
        batch[1]['chain_lens'] = [chain_lens.unsqueeze(0), chain_lens_mask]
        with torch.no_grad():
            with autocast(dtype=torch.float16):
                output_dict = feature_getter(trained_model, tokenizer, batch, pdb=None, weight_dists=weight_dists, seq_mask=seq_mask, focus=focus, top_k=top_k, struct_mask=struct_mask, remove_far=remove_far, threshold=threshold, dev=dev)
                score, scores, plot_scores, plot_weights, plot_pep_mask, plot_indices, plot_X, plot_seq = compute_score(output_dict, weight_dists, MAX_LEN, plot_scores=plot_scores, plot_weights=plot_weights, plot_pep_mask=plot_pep_mask, plot_indices=plot_indices, plot_X=plot_X, plot_seq=plot_seq, is_complex=True)
                scores_stats['models'].append(model)
                scores_stats['seqs'].append(pep_seq)
                scores_stats['rla_scores'].append(score)
                paired_res_scores[model] = score

running on 4cay.
{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': 0, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0


1it [00:00, 16.03it/s]
0it [00:00, ?it/s]/tmp/ipykernel_255123/3490658105.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):
1it [00:00,  5.75it/s]

running on 4cay_new.
{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': 0, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0



1it [00:00, 13.56it/s]
1it [00:00,  5.91it/s]

running on 4cay_supercloud.
{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': 0, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0



1it [00:00, 14.65it/s]
1it [00:00,  6.02it/s]


In [15]:
scores_stats

{'models': ['pdbs/4cay', 'pdbs/4cay', 'pdbs/4cay'],
 'seqs': ['SRSQRAGLQFPVGRIHRHLKSRGRVGATAAVYSAAILEYLTAEVLELAGNASKDLKVKRITPRHLQLAIRGDEELDSLIKATIAGG',
  'SRSQRAGLQFPVGRIHRHLKSRGRVGATAAVYSAAILEYLTAEVLELAGNASKDLKVKRITPRHLQLAIRGDEELDSLIKATIAGG',
  'SRSQRAGLQFPVGRIHRHLKSRGRVGATAAVYSAAILEYLTAEVLELAGNASKDLKVKRITPRHLQLAIRGDEELDSLIKATIAGG'],
 'rla_scores': [0.30594228208065033, 0.30594228208065033, 0.30594228208065033]}

In [ ]:
## SAVE/LOAD RAW SCORES AND SEQUENCE MAPPINGS (CHANGE JSON WRITING/READING AS NEEDED)
import json

with open('/PATH/TO/OUTPUTS/seq_mapping.json', 'w') as f:
    json.dump(scores_stats, f)
with open('/PATH/TO/OUTPUTS/scores.json', 'w') as f:
    json.dump(paired_res_scores, f)